In [1]:
import os

In [2]:
%pwd

'/home/tejas/MLProjects/Thyroid-Disease-Prediction/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/tejas/MLProjects/Thyroid-Disease-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from ThyroidProject import logger
from ThyroidProject.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(
                f"{filename} downloaded! with the following information: \n{headers}")
        else:
            logger.info(
                f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None 
        """
        unzip_path = self.config.unzip_dir
        filename = "thyroid0387.data"
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extract(filename, unzip_path)
            os.rename(os.path.join(unzip_path, filename) , os.path.join(unzip_path,"data.csv"))

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-22 00:18:24,323:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-01-22 00:18:24,324:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-22 00:18:24,325:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-01-22 00:18:24,326:INFO:common:created directory at :artifacts]
[2024-01-22 00:18:24,326:INFO:common:created directory at :artifacts/data_ingestion]
[2024-01-22 00:18:28,453:INFO:2590503970:artifacts/data_ingestion/data.zip downloaded! with the following information: 
Server: nginx/1.25.3
Date: Sun, 21 Jan 2024 18:48:25 GMT
Transfer-Encoding: chunked
Connection: close

]
